# Calculating the difference of training speed between my personal CPU and GPU
 - CPU: Intel Core i7-H8750 2.2GHz 12CPUs
 - GPU: NVIDIA GeForce GTX 1050 Ti 4GB Driver 512.78
 
## The model trained:
 - 3 * (2D Convolutional, Batch Normalization, Max Pooling and Flatten for the last one) and 2 Dense layers
 - Total trainable parameteres: 226K
 - Batch Size: 64
 - Epochs: 10
 - Optimizer: Adam

In [5]:
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [3]:
def build_and_train_model(device):
    with tf.device(device):
        # Define the model
        inputs = Input(shape=(32, 32, 3))
        x = Conv2D(32, 3)(inputs)
        x = BatchNormalization()(x)
        x = tf.nn.relu(x)
        x = MaxPooling2D()(x)
        x = Conv2D(64, 3)(x)
        x = BatchNormalization()(x)
        x = tf.nn.relu(x)
        x = MaxPooling2D()(x)
        x = Conv2D(128, 3)(x)
        x = BatchNormalization()(x)
        x = tf.nn.relu(x)
        x = Flatten()(x)
        x = Dense(64, activation="relu")(x)
        outputs = Dense(10)(x)
        model = tf.keras.Model(inputs=inputs, outputs=outputs)

        # Compile the model
        model.compile(
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=Adam(),
            metrics=["accuracy"],
        )

        # Measure training time
        start_time = time.time()
        history = model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
        end_time = time.time()

        # Calculate elapsed time
        training_time = end_time - start_time
        print(f"Training time on {device}: {training_time:.2f} seconds")

        # Evaluate the model
        test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=64, verbose=2)
        print(f"Test Accuracy on {device}: {test_acc:.2f}")

    return training_time, test_acc

In [6]:
cpu_time, cpu_accuracy = build_and_train_model('/CPU:0')
gpu_time, gpu_accuracy = build_and_train_model('/GPU:0')

print(f"CPU Training Time: {cpu_time:.2f} seconds, Accuracy: {cpu_accuracy:.2f}")
print(f"GPU Training Time: {gpu_time:.2f} seconds, Accuracy: {gpu_accuracy:.2f}")

Epoch 1/10
782/782 - 53s - loss: 1.2623 - accuracy: 0.5483 - 53s/epoch - 68ms/step
Epoch 2/10
782/782 - 46s - loss: 0.8748 - accuracy: 0.6927 - 46s/epoch - 59ms/step
Epoch 3/10
782/782 - 45s - loss: 0.7368 - accuracy: 0.7411 - 45s/epoch - 58ms/step
Epoch 4/10
782/782 - 45s - loss: 0.6416 - accuracy: 0.7761 - 45s/epoch - 58ms/step
Epoch 5/10
782/782 - 45s - loss: 0.5652 - accuracy: 0.8028 - 45s/epoch - 58ms/step
Epoch 6/10
782/782 - 45s - loss: 0.4999 - accuracy: 0.8257 - 45s/epoch - 58ms/step
Epoch 7/10
782/782 - 45s - loss: 0.4366 - accuracy: 0.8477 - 45s/epoch - 58ms/step
Epoch 8/10
782/782 - 46s - loss: 0.3849 - accuracy: 0.8654 - 46s/epoch - 59ms/step
Epoch 9/10
782/782 - 45s - loss: 0.3344 - accuracy: 0.8817 - 45s/epoch - 57ms/step
Epoch 10/10
782/782 - 45s - loss: 0.2902 - accuracy: 0.8975 - 45s/epoch - 58ms/step
Training time on /CPU:0: 470.19 seconds
157/157 - 2s - loss: 0.9719 - accuracy: 0.7326 - 2s/epoch - 16ms/step
Test Accuracy on /CPU:0: 0.73
Epoch 1/10
782/782 - 21s - lo

# Results

CPU Training Time: 470.2 seconds

GPU Training Time: 86.3 seconds

Ratio: 5.4 times faster